# Определяем перспективный тариф для телеком-компании «Мегалайн»

In [2]:
import pandas as pd
import scipy.stats as st
import matplotlib as mpl
import seaborn as sns

<div class="alert alert-info" role="alert">
    <p class="mb-0">В коде использую <code>display()</code>, потомучто это однозначно даёт понять что планируется вывод / печать результатов. То есть использую <code>display()</code> намеренно, хотя понимаю что можно и без него выводить данные. 👌</p>
</div>

In [13]:
calls_df = pd.read_csv('./datasets/calls.csv')
internet_df = pd.read_csv('./datasets/internet.csv', index_col=0)
messages_df = pd.read_csv('./datasets/messages.csv')
tariffs_df = pd.read_csv('./datasets/tariffs.csv')
users_df = pd.read_csv('./datasets/users.csv')

In [11]:
display(
    calls_df.sample(3),
    calls_df.describe()
)

,id,call_date,duration,user_id
128848,1330_712,2018-08-28,2.31,1330
38818,1097_150,2018-11-07,24.71,1097
100879,1260_43,2018-09-27,17.63,1260


,duration,user_id
count,202607.000000,202607.000000
mean,6.755887,1253.940619
std,5.843365,144.722751
min,0.000000,1000.000000
25%,1.300000,1126.000000
50%,6.000000,1260.000000
75%,10.700000,1379.000000
max,38.000000,1499.000000


In [14]:
display(
    internet_df.sample(3),
    internet_df.describe()
)

,id,mb_used,session_date,user_id
39410,1138_88,456.01,2018-08-02,1138
4630,1014_236,83.36,2018-10-22,1014
86355,1292_471,1059.20,2018-05-08,1292


,mb_used,user_id
count,149396.000000,149396.000000
mean,370.192426,1252.099842
std,278.300951,144.050823
min,0.000000,1000.000000
25%,138.187500,1130.000000
50%,348.015000,1251.000000
75%,559.552500,1380.000000
max,1724.830000,1499.000000


In [15]:
display(
    messages_df.sample(3),
    messages_df.describe()
)

,id,message_date,user_id
110317,1447_2,2018-09-30,1447
45308,1196_758,2018-06-18,1196
50805,1226_481,2018-09-11,1226


,user_id
count,123036.000000
mean,1256.989410
std,143.523967
min,1000.000000
25%,1134.000000
50%,1271.000000
75%,1381.000000
max,1499.000000


In [9]:
display(
    tariffs_df
)

,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


In [16]:
display(
    users_df.sample(3),
    users_df.describe()
)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
23,1023,42,NaN,Томск,Людмила,Мельникова,2018-10-22,smart
300,1300,40,NaN,Москва,Тихон,Азерников,2018-07-26,ultra
307,1307,53,NaN,Самара,Симона,Авсейкина,2018-05-04,smart


,user_id,age
count,500.000000,500.00000
mean,1249.500000,46.58800
std,144.481833,16.66763
min,1000.000000,18.00000
25%,1124.750000,32.00000
50%,1249.500000,46.00000
75%,1374.250000,62.00000
max,1499.000000,75.00000


## Шаг 0. Введение

В этом учебном проекте сделаем предварительный анализ тарифов на выборке данных 500 пользователей «Мегалайна». Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

<hr>

## Шаг 1. Знакомимся с данными изучаем общую информацию

Откройте файл с данными и изучите общую информацию

<hr>

## Шаг 2. Подготавливаем данные

### Найдите и исправьте ошибки в данных, если они есть.

#### Поясните, какие ошибки вы нашли и как их исправили.

### Приведите данные к нужным типам;

<hr>

### Посчитайте для каждого пользователя:

#### количество сделанных звонков и израсходованных минут разговора по месяцам;

#### количество отправленных сообщений по месяцам;

#### объем израсходованного интернет-трафика по месяцам;

#### помесячную выручку с каждого пользователя

(вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату, соответствующую тарифному плану).

<hr>

## Шаг 3. Анализ данных

### Описание поведения клиентов оператора, исходя из выборки.

#### Сколько минут разговора требуется пользователям каждого тарифа в месяц?

 ##### Среднее количество минут, дисперсия и стандартное отклонение.

##### Гистограммы минут разговора.

##### Описание распределений минут разговора.

<hr>

#### Сколько сообщений требуется пользователям каждого тарифа в месяц?

 ##### Среднее количество сообщений, дисперсия и стандартное отклонение.

##### Гистограммы сообщений.

##### Описание распределений сообщений.

<hr>

#### Какой объём интернет-трафика требуется пользователям каждого тарифа в месяц?

 Посчитайте среднее количество интернет-трафика, дисперсию и стандартное отклонение.

##### Гистограммы интернет-трафика.

##### Описание распределений интернет-трафика.

<hr>

## Шаг 4. Проверьте гипотезы

### Средняя выручка пользователей тарифов «Ультра» и «Смарт» различаются

### Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов

Поясните:
как вы формулировали нулевую и альтернативную гипотезы;
какой критерий использовали для проверки гипотез и почему.

<hr>

## Шаг 5. Общий вывод

Описание данных
Таблица users (информация о пользователях):
user_id — уникальный идентификатор пользователя
first_name — имя пользователя
last_name — фамилия пользователя
age — возраст пользователя (годы)
reg_date — дата подключения тарифа (день, месяц, год)
churn_date — дата прекращения пользования тарифом (если значение пропущено, то тариф ещё действовал на момент выгрузки данных)
city — город проживания пользователя
tariff — название тарифного плана
Таблица calls (информация о звонках):
id — уникальный номер звонка
call_date — дата звонка
duration — длительность звонка в минутах
user_id — идентификатор пользователя, сделавшего звонок
Таблица messages (информация о сообщениях):
id — уникальный номер сообщения
message_date — дата сообщения
user_id — идентификатор пользователя, отправившего сообщение
Таблица internet (информация об интернет-сессиях):
id — уникальный номер сессии
mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах)
session_date — дата интернет-сессии
user_id — идентификатор пользователя
Таблица tariffs (информация о тарифах):
tariff_name — название тарифа
rub_monthly_fee — ежемесячная абонентская плата в рублях
minutes_included — количество минут разговора в месяц, включённых в абонентскую плату
messages_included — количество сообщений в месяц, включённых в абонентскую плату
mb_per_month_included — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
rub_per_minute — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
rub_per_message — стоимость отправки сообщения сверх тарифного пакета
rub_per_gb — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)